Cryptocurrency trading bot.

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

# File:        cryptocurrency/crypto_logger_base.py
# By:          Samuel Duclos
# For          Myself
# Description: Simple Binance logger base class.

# Library imports.
from cryptocurrency.resampling import resample
from binance.client import Client
from abc import abstractmethod, ABC
from time import sleep, time
from os.path import exists, join
from os import mkdir

import pandas as pd

class Crypto_logger_base(ABC):
    def __init__(self, interval='15s', delay=4.7, buffer_size=3000, directory='crypto_logs', 
                 log_name='crypto_log', raw=False):
        """
        :param interval: OHLCV interval to log. Default is 15 seconds.
        :param delay: delay between Binance API requests. Minimum calculated was 4.7 seconds.
        :param buffer_size: buffer size to avoid crashing on memory accesses.
        :param directory: the directory where to output the logs.
        :param log_name: name of the log file.
        :param raw: whether the log dumps raw (instantaneous) or OHLCV data.
        """
        self.interval = interval
        self.delay = delay
        self.buffer_size = buffer_size
        self.directory = directory
        self.raw = raw

        self.log_name = join(self.directory, log_name + '.txt')
        self.log_screened_name = join(self.directory, log_name + '_screened.txt')

        if not exists(self.directory):
            mkdir(self.directory)

    #self.get_from_file(log_name=self.log_name, from_raw=False)
    #self.get_from_file(log_name=self.input_log_name, from_raw=self.load_from_ohlcv)
    def get_from_file(self, log_name, from_raw=False):
        if from_raw:
            dataset = pd.read_csv(log_name, header=0, index_col=0)
        else:
            dataset = pd.read_csv(log_name, header=[0, 1], index_col=0)
        dataset.index = pd.DatetimeIndex(dataset.index)
        return dataset.sort_index(axis='index')

    @abstractmethod
    def get(self, **kwargs):
        raise NotImplementedError()

    @abstractmethod
    def screen(self, **kwargs):
        raise NotImplementedError()

    def put(self, dataset):
        dataset = dataset.copy().reset_index()
        if self.raw:
            dataset = dataset.drop_duplicates(subset=['symbol', 'count'], 
                                              keep='first', ignore_index=True)
        else:
            dataset = dataset.drop_duplicates(keep='last', ignore_index=True)

        if 'date' in dataset.columns:
            min_index_int = dataset[dataset['date'] == self.min_index].index[0]
            dataset = dataset.set_index('date')
        if not self.raw:
            dataset = resample(dataset, self.interval)
        if 'date' in dataset.columns:
            dataset = dataset.iloc[min_index_int:]

        dataset = dataset.tail(self.buffer_size)
        dataset.to_csv(self.log_name)
        self.min_index = dataset.index[0]
        return dataset

    def start(self, append=False, roll=0):
        """Main logger loop."""
        print('Starting crypto logger.')

        if exists(self.log_name) and 'output' in self.log_name:
            self.dataset = self.get_from_file(log_name=self.log_name, from_raw=False)
            self.dataset = self.dataset.tail(self.buffer_size)
        else:
            self.dataset = self.get()

        self.min_index = self.dataset.index[-1]
        self.dataset = self.put(self.dataset)

        while True:
            try:
                dataset = pd.concat([self.dataset, self.get()], axis='index', join='outer')
            except (KeyboardInterrupt, SystemExit):
                print('User terminated crypto logger process.')
                break
            except Exception as e:
                print(e)
            try:
                self.dataset = self.put(dataset)
            except (KeyboardInterrupt, SystemExit):
                print('Saving latest complete dataset...')
                self.dataset = self.put(dataset)
                print('User terminated crypto logger process.')
                break
            except Exception as e:
                print(e)
            try:
                if exists(self.log_screened_name):
                    dataset_screened_old = \
                        pd.read_csv(self.log_screened_name, index_col=0, header=0)
                else:
                    dataset_screened_old = None
                dataset_screened = self.screen(self.dataset)
                if dataset_screened is not None:
                    if roll != 0:
                        if append and exists(self.log_screened_name):
                            dataset_screened = \
                                pd.concat([dataset_screened_old, dataset_screened], axis='index')
                            dataset_screened = \
                                dataset_screened.drop_duplicates(subset=['symbol'], keep='last')
                        dataset_screened = dataset_screened.tail(roll)
                        dataset_screened.to_csv(self.log_screened_name)
                    elif append:
                        dataset_screened.to_csv(self.log_screened_name, mode='a')
                    else:
                        dataset_screened.to_csv(self.log_screened_name)
            except (KeyboardInterrupt, SystemExit):
                print('User terminated crypto logger process.')
                break
            except Exception as e:
                print(e)
            sleep(self.delay)
        print('Crypto logger process done.')

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

# File:        cryptocurrency/crypto_logger_output.py
# By:          Samuel Duclos
# For          Myself
# Description: Simple Binance logger output for arbitrary intervals.

# Library imports.
#from cryptocurrency.crypto_logger_base import Crypto_logger_base
from cryptocurrency.indicators import filter_in_market, screen_one
from os.path import exists, join

import pandas as pd
pd.options.mode.chained_assignment = None

class Crypto_logger_output(Crypto_logger_base):
    def __init__(self, delay=10, interval_input='15s', interval='15s', buffer_size=100, 
                 input_log_name='input'):
        """
        :param delay: delay between Binance API requests. Minimum calculated was 4.7 seconds.
        :param interval_input: OHLCV interval from input log. Default is 15 seconds.
        :param interval: OHLCV interval to log. Default is 15 seconds.
        :param buffer_size: buffer size to avoid crashing on memory accesses.
        :param input_log_name: the directory where to take the logs from.
        """
        self.data_before = pd.DataFrame()
        input_log_name = 'crypto_' + input_log_name + '_log_'
        self.load_from_ohlcv = interval_input != interval
        super().__init__(interval=interval, delay=delay, buffer_size=buffer_size, 
                         directory='crypto_logs', log_name='crypto_output_log_' + interval, 
                         raw=False)

        self.input_log_name = \
            join(self.directory, input_log_name + interval_input + '.txt')
        self.input_log_screened_name = \
            join(self.directory, input_log_name + interval_input + '_screened.txt')

    def screen(self, dataset):
        if exists(self.input_log_screened_name):
            input_filtered = pd.read_csv(self.input_log_screened_name, header=0, index_col=0)
            input_filter = set(input_filtered['symbol'].tolist())
            old_columns = set(dataset.columns.get_level_values(0).tolist())
            new_columns = list(input_filter & old_columns)
            dataset = dataset[new_columns]
            #dataset.columns = dataset.columns.swaplevel(0, 1)
            #dataset = dataset.rename(columns={'base_volume': 'volume'})
            #dataset.columns = dataset.columns.swaplevel(0, 1)
            assets = filter_in_market(screen_one, dataset)
            #dataset.columns = dataset.columns.swaplevel(0, 1)
            #dataset = dataset.rename(columns={'volume': 'base_volume'})
            #dataset.columns = dataset.columns.swaplevel(0, 1)
            return input_filtered[input_filtered['symbol'].isin(assets)]
        else:
            return None

    def resample_from_raw(self, df):
        df = df[['symbol', 'close', 'rolling_base_volume', 'rolling_quote_volume']]
        df['base_volume'] = df['rolling_base_volume'].copy()
        df['quote_volume'] = df['rolling_quote_volume'].copy()
        df = df.pivot_table(index=['date'], columns=['symbol'], 
                            values=['close', 'rolling_base_volume', 
                                    'rolling_quote_volume', 
                                    'base_volume', 'quote_volume'], 
                            aggfunc={'close': ['first', 'max', 'min', 'last'], 
                                     'base_volume': 'max', 'quote_volume': 'max', 
                                     'rolling_base_volume': 'max', 
                                     'rolling_quote_volume': 'max'})
        df.columns = pd.MultiIndex.from_tuples([('_'.join(col[:2]), col[2]) for col in df.columns.values], 
                                               names=('pair', 'symbol'))
        df = df.rename(columns={'close_first': 'open', 'close_max': 'high', 
                                'close_min': 'low', 'close_last': 'close', 
                                'base_volume_max': 'base_volume', 
                                'quote_volume_max': 'quote_volume', 
                                'rolling_base_volume_max': 'rolling_base_volume', 
                                'rolling_quote_volume_max': 'rolling_quote_volume'}, 
                       level=0)
        df['base_volume'] = df['base_volume'].fillna(method='pad')
        df['base_volume'].iloc[0] = 0
        df['quote_volume'] = df['quote_volume'].fillna(method='pad')
        df['quote_volume'].iloc[0] = 0
        df['rolling_base_volume'] = df['rolling_base_volume'].fillna(method='pad')
        df['rolling_base_volume'].iloc[0] = 0
        df['rolling_quote_volume'] = df['rolling_quote_volume'].fillna(method='pad')
        df['rolling_quote_volume'].iloc[0] = 0
        df = df.sort_index().iloc[1:]
        df.columns = df.columns.swaplevel(0, 1)
        return df

    def get(self):
        dataset = self.get_from_file(log_name=self.input_log_name, 
                                     from_raw=not self.load_from_ohlcv)
        if not self.load_from_ohlcv:
            dataset = self.resample_from_raw(dataset)
        return dataset.tail(2)

Main.

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

# File:        crypto_logger_output_1min.py
# By:          Samuel Duclos
# For          Myself
# Description: Simple Binance logger output for the 1 minute interval.

# Library imports.
#from cryptocurrency.crypto_logger_output import Crypto_logger_output

crypto_logger_output_1min = Crypto_logger_output(delay=44, 
                                                 interval_input='30s', 
                                                 interval='1min', 
                                                 buffer_size=1500, 
                                                 input_log_name='output')
crypto_logger_output_1min.start(append=False, roll=1000)

In [ ]:
import pandas as pd
input_log_name = 'crypto_logs/crypto_input_log_15s.txt'
df = pd.read_csv(input_log_name, header=0, index_col=0)
df.index = pd.DatetimeIndex(df.index)
df = df.sort_index(axis='index')
#df = crypto_logger_output_1min.resample_from_raw(df)
df

In [ ]:
import pandas as pd
#crypto_output_log_1min = crypto_logger_output_1min.log_name
crypto_output_log_1min = 'crypto_logs/crypto_output_log_1min.txt'
df_1min = pd.read_csv(crypto_output_log_1min, header=[0, 1], index_col=0)
df_1min.index = pd.DatetimeIndex(df_1min.index)
df_1min = df_1min.sort_index(axis='index')
df_1min['BTCUSDT'].tail(20)

In [ ]:
dataset = crypto_logger_output_1min.dataset
dataset

In [ ]:
dataset['BTCUSDT']

In [ ]:
dataset['ZILBUSD']

In [ ]:
def filter_in_market(function, dataset):
    def f(x):
        x = x.loc[:,~x.columns.duplicated()]
        return function(x)
    tickers_list = dataset.columns.get_level_values(0).unique().tolist()
    return pd.Series([ticker for ticker in tickers_list if f(dataset[ticker])], dtype='str')

def get_positive_trend_strength_trigger(data):
    ADX = data.ta.adx(talib=True)
    return (ADX['ADX_14'] < 0.20).iloc[-3] & (ADX['ADX_14'] > 0.20).iloc[-2]

def get_not_negative_trend_strength_trigger(data):
    ADX = data.ta.adx(length=14, lensig=8, talib=True)
    return ((ADX['DMP_14'] > ADX['DMN_14']) & (ADX['ADX_14'] > 0.30)).iloc[-1]

def get_not_negative_rebound_trigger(data):
    CCI = data.ta.cci(length=22, talib=True)
    MFI = data.ta.mfi(length=11, talib=True)
    return ((CCI > 0) | (MFI > 20)).iloc[-1]

def get_positive_choppiness_trigger(data):
    CHOP = data.ta.chop(talib=True)
    return CHOP.iloc[-1] < 38.2

def get_positive_phase_trigger(data):
    MACD = data.ta.macd(talib=True)
    histogram = MACD['MACDs_12_26_9'] - MACD['MACD_12_26_9']
    return ((histogram > histogram.shift(1)) | \
            (MACD['MACD_12_26_9'] > MACD['MACDs_12_26_9'])).iloc[-1]

def get_positive_phase_trigger(data):
    MACD = data.ta.macd(talib=True)
    histogram = MACD['MACDs_12_26_9'] - MACD['MACD_12_26_9']
    return ((histogram.iloc[-2] > histogram.iloc[-2]) or \
            (MACD['MACD_12_26_9'].iloc[-1] > MACD['MACDs_12_26_9'].iloc[-1]))

In [ ]:
filter_in_market(get_positive_phase_trigger, df)

In [ ]:
get_positive_phase_trigger(df['BTCUSDT'])

In [ ]:
from tqdm import tqdm

def filter_in_market(function, dataset):
    def f(x):
        x = x.loc[:,~x.columns.duplicated()]
        return function(x)
    tickers_list = dataset.columns.get_level_values(0).unique().tolist()
    return pd.Series([ticker for ticker in tqdm(tickers_list, unit=' ticker') if f(dataset[ticker])], dtype='str')

def get_heikin_ashi_trigger(ticker):
    def get_trend_strength_entry_trigger(heikin_ashi_dataset):
        ADX = heikin_ashi_dataset.ta.adx(talib=True)
        return (ADX['ADX_14'] < 0.20).iloc[-3] and (ADX['ADX_14'] > 0.20).iloc[-2]

    def get_not_negative_rebound_trigger(heikin_ashi_dataset):
        CCI = heikin_ashi_dataset.ta.cci(talib=True)
        return CCI.iloc[-1] < 100

    def get_positive_choppiness_trigger(heikin_ashi_dataset):
        CHOP = heikin_ashi_dataset.ta.chop(talib=True)
        return CHOP.iloc[-1] < 38.2

    def get_buy_trigger(heikin_ashi_dataset):
        return get_not_negative_rebound_trigger(heikin_ashi_dataset) and \
               (get_positive_choppiness_trigger(heikin_ashi_dataset) or \
                get_trend_strength_entry_trigger(heikin_ashi_dataset))

    heikin_ashi = ticker.ta.ha(talib=True)
    heikin_ashi_dataset = heikin_ashi.rename(columns={'HA_open': 'open', 
                                                      'HA_high': 'high', 
                                                      'HA_low': 'low', 
                                                      'HA_close': 'close'})
    try:
        trigger = get_buy_trigger(heikin_ashi_dataset)
    except:
        trigger = False
    return trigger

tickers_list = filter_in_market(get_heikin_ashi_trigger, dataset)
tickers_list

In [ ]:
heikin_ashi = dataset['BTCUSDT'].ta.ha(talib=True)
heikin_ashi_dataset = heikin_ashi.rename(columns={'HA_open': 'open', 
                                                  'HA_high': 'high', 
                                                  'HA_low': 'low', 
                                                  'HA_close': 'close'})
ADX = heikin_ashi_dataset.ta.adx(talib=True)
ADX[['DMP_14', 'DMN_14']]

In [ ]:
import pandas as pd
crypto_output_log_15s = 'crypto_logs/crypto_output_log_15s.txt'
df_15s = pd.read_csv(crypto_output_log_15s, header=[0, 1], index_col=0)
df_15s.index = pd.DatetimeIndex(df_15s.index)
df_15s

In [ ]:
import pandas as pd

crypto_exchange_info = 'crypto_logs/crypto_exchange_info.txt'
exchange_info = pd.read_csv(crypto_exchange_info, header=0, index_col=0)
exchange_info

In [ ]:
from cryptocurrency.authentication import Cryptocurrency_authenticator
from cryptocurrency.exchange import Cryptocurrency_exchange
from cryptocurrency.conversion_table import get_conversion_table

authenticator = Cryptocurrency_authenticator(use_keys=False, testnet=False)
client = authenticator.spot_client
exchange = Cryptocurrency_exchange(client=client, directory='crypto_logs')
exchange_info = exchange.info

#conversion_table = get_conversion_table(client=client, exchange_info=exchange_info)
#conversion_table.sort_values(by='rolling_traded_volume', ascending=False).reset_index(drop=True).head(50)

In [ ]:
from cryptocurrency.conversion import convert_price, get_base_asset_from_pair, get_quote_asset_from_pair
from pandas import concat, DataFrame

def get_conversion_table(client, exchange_info):
    """
    Fetches and prepares data used to calculate prices, volumes and other stats.
    :param client: object from python-binance useful for calling client.get_ticker().
    :param exchange_info: Pre-calculated exchange information on all tickers.
    :return: pd.DataFrame containing all preprocessed conversion table info.
    :column is_shorted: is the symbol made from inversion.
    :column symbol: concatenated string made from base_asset and quote_asset.
    :column shorted_symbol: symbol with inverted base_asset and quote_asset.
    :column base_asset: asset on the left.
    :column quote_asset: asset on the right.
    :column price_change: (close - open).
    :column price_change_percent: ((close - open) / open).
    :column weighted_average_price: weighted average price.
    :column close_shifted: close price of the previous day.
    :column open: open price of the day.
    :column high: high price of the day.
    :column low: low price of the day.
    :column close: close price of the day.
    :column last_volume: volume of the last price update.
    :column USDT_bid_price: USDT-converted bid price.
    :column USDT_ask_price: USDT-converted ask price.
    :column USDT_bid_volume: USDT-converted bid volume.
    :column USDT_ask_volume: USDT-converted ask volume.
    :column bid_price: price of the bid.
    :column bid_volume: volume of the bid at bid_price.
    :column ask_price: price of the ask.
    :column ask_volume: volume of the ask at ask_price.
    :column rolling_base_volume: rolling_base_volume given by the API.
    :column rolling_quote_volume: rolling_quote_volume given by the API.
    :column open_time: close_time minus 24 hours.
    :column close_time: time from epoch in milliseconds of the last price update.
    :column first_ID: transaction ID from 1 day ago.
    :column last_ID: latest transaction ID.
    :column count: value calculated by subtracting first_ID from last_ID.
    :column USDT_price: value calculated by subtracting firstId from lastId.
    :column rolling_USDT_base_volume: USDT-converted rolling_base_volume.
    :column rolling_USDT_quote_volume: USDT-converted rolling_quote_volume.
    :column rolling_traded_volume: sum by base_asset of all USDT-converted volumes.
    :column importance: rolling_USDT_base_volume divided by rolling_traded_volume.
    :column traded_price: sum by base_asset of all (close prices times importance).
    :column traded_bid_price: sum by base_asset of all (bid prices times importance).
    :column traded_ask_price: sum by base_asset of all (ask prices times importance).
    :column bid_ask_change_percent: ((ask_price - bid_price) / ask_price) * 100).
    :column bid_ask_volume_percent: ((bid_volume / (bid_volume + ask_volume)) * 100).
    :column traded_bid_ask_change_percent: ((traded_ask_price - traded_bid_price) / traded_ask_price) * 100).
    :column traded_bid_ask_volume_percent: ((traded_bid_volume / (traded_bid_volume + traded_ask_volume)) * 100).
    """
    conversion_table = DataFrame(client.get_ticker())

    conversion_table = conversion_table[conversion_table['symbol'].isin(exchange_info['symbol'])]
    conversion_table['base_asset'] = \
        conversion_table['symbol'].apply(lambda x: get_base_asset_from_pair(x, exchange_info=exchange_info))
    conversion_table['quote_asset'] = \
        conversion_table['symbol'].apply(lambda x: get_quote_asset_from_pair(x, exchange_info=exchange_info))

    conversion_table = \
        conversion_table.rename(columns={'openPrice': 'open', 'highPrice': 'high', 'lowPrice': 'low', 
                                         'lastPrice': 'close', 'volume': 'rolling_base_volume', 
                                         'quoteVolume': 'rolling_quote_volume', 'lastQty': 'last_volume', 
                                         'bidPrice': 'bid_price', 'askPrice': 'ask_price', 
                                         'bidQty': 'bid_volume', 'askQty': 'ask_volume', 
                                         'firstId': 'first_ID', 'lastId': 'last_ID', 
                                         'openTime': 'open_time', 'closeTime': 'close_time', 
                                         'prevClosePrice': 'close_shifted', 
                                         'weightedAvgPrice': 'weighted_average_price', 
                                         'priceChange': 'price_change', 
                                         'priceChangePercent': 'price_change_percent'})
    conversion_table[['price_change_percent', 'open', 'high', 'low', 'close', 'close_shifted', 'bid_price', 
                      'ask_price', 'bid_volume', 'ask_volume', 'rolling_base_volume', 'rolling_quote_volume', 
                      'count']] = \
        conversion_table[['price_change_percent', 'open', 'high', 'low', 'close', 'close_shifted', 'bid_price', 
                          'ask_price', 'bid_volume', 'ask_volume', 'rolling_base_volume', 'rolling_quote_volume', 
                          'count']].astype(float)

    conversion_table['rolling_base_quote_volume'] = \
        conversion_table['rolling_quote_volume'] / conversion_table['close']
    conversion_table['USDT_price'] = \
        conversion_table.apply(lambda x: convert_price(size=1, from_asset=x['base_asset'], to_asset='USDT', 
                                                       conversion_table=conversion_table, 
                                                       exchange_info=exchange_info), axis='columns')
    conversion_table['rolling_USDT_base_volume'] = \
        conversion_table['rolling_base_volume'] * conversion_table['USDT_price']
    conversion_table['rolling_USDT_quote_volume'] = \
        conversion_table['rolling_base_quote_volume'] * conversion_table['USDT_price']

    conversion_table['USDT_bid_price'] = \
        conversion_table.apply(lambda x: convert_price(size=x['bid_price'], from_asset=x['base_asset'], 
                                                       to_asset='USDT', conversion_table=conversion_table, 
                                                       exchange_info=exchange_info), 
                               axis='columns')
    conversion_table['USDT_ask_price'] = \
        conversion_table.apply(lambda x: convert_price(size=x['ask_price'], from_asset=x['base_asset'], 
                                                       to_asset='USDT', conversion_table=conversion_table, 
                                                       exchange_info=exchange_info), 
                               axis='columns')
    conversion_table['USDT_bid_volume'] = \
        conversion_table['bid_volume'] * conversion_table['USDT_bid_price']
    conversion_table['USDT_ask_volume'] = \
        conversion_table['ask_volume'] * conversion_table['USDT_ask_price']

    conversion_table['is_shorted'] = False

    conversion_table_swapped = conversion_table.copy()
    conversion_table_swapped.loc[:, ['symbol', 'price_change', 'price_change_percent', 
                                     'weighted_average_price', 'close_shifted', 'close', 
                                     'last_volume', 'ask_price', 'ask_volume', 'bid_price', 
                                     'bid_volume', 'open', 'high', 'low', 'rolling_quote_volume', 
                                     'rolling_base_volume', 'open_time', 'close_time', 'first_ID', 
                                     'last_ID', 'count', 'quote_asset', 'base_asset', 
                                     'rolling_base_quote_volume', 'USDT_price', 
                                     'rolling_USDT_quote_volume', 'rolling_USDT_base_volume', 
                                     'USDT_ask_price', 'USDT_bid_price', 'USDT_ask_volume', 
                                     'USDT_bid_volume', 'is_shorted']] = \
        conversion_table_swapped.loc[:, ['symbol', 'price_change', 'price_change_percent', 'weighted_average_price', 
                                         'close_shifted', 'close', 'last_volume', 'bid_price', 'bid_volume', 
                                         'ask_price', 'ask_volume', 'open', 'high', 'low', 'rolling_base_volume', 
                                         'rolling_quote_volume', 'open_time', 'close_time', 'first_ID', 'last_ID', 
                                         'count', 'base_asset', 'quote_asset', 'rolling_base_quote_volume', 
                                         'USDT_price', 'rolling_USDT_base_volume', 'rolling_USDT_quote_volume', 
                                         'USDT_bid_price', 'USDT_ask_price', 'USDT_bid_volume', 'USDT_ask_volume', 
                                         'is_shorted']].values
    conversion_table_swapped['symbol'] = \
        conversion_table_swapped['base_asset'] + conversion_table_swapped['quote_asset']
    conversion_table_swapped.loc[:, ['open', 'high', 'low', 'close', 'close_shifted', 'bid_price', 'ask_price', 
                                     'USDT_price', 'USDT_bid_price', 'USDT_ask_price']] = \
        1 / conversion_table_swapped.loc[:, ['open', 'high', 'low', 'close', 'close_shifted', 
                                             'bid_price', 'ask_price', 'USDT_price', 
                                             'USDT_bid_price', 'USDT_ask_price']].astype(float)
    conversion_table_swapped['is_shorted'] = True

    conversion_table = concat([conversion_table, conversion_table_swapped], join='outer', axis='index')

    traded_volume = conversion_table.groupby(by='base_asset').agg('sum')
    traded_volume = traded_volume['rolling_USDT_base_volume']
    conversion_table['rolling_traded_volume'] = \
        conversion_table.apply(lambda x: traded_volume.loc[x['base_asset']], axis='columns')
    traded_bid_volume = conversion_table.groupby(by='base_asset').agg('sum')
    traded_bid_volume = traded_bid_volume['USDT_bid_volume']
    conversion_table['traded_bid_volume'] = \
        conversion_table.apply(lambda x: traded_bid_volume.loc[x['base_asset']], axis='columns')
    traded_ask_volume = conversion_table.groupby(by='base_asset').agg('sum')
    traded_ask_volume = traded_ask_volume['USDT_ask_volume']
    conversion_table['traded_ask_volume'] = \
        conversion_table.apply(lambda x: traded_ask_volume.loc[x['base_asset']], axis='columns')

    conversion_table['importance'] = \
        conversion_table['rolling_USDT_base_volume'] / conversion_table['rolling_traded_volume']

    conversion_table['importance_weighted_price'] = \
        conversion_table['USDT_price'] * conversion_table['importance']
    conversion_table['importance_weighted_bid_price'] = \
        conversion_table['USDT_bid_price'] * conversion_table['importance']
    conversion_table['importance_weighted_ask_price'] = \
        conversion_table['USDT_ask_price'] * conversion_table['importance']

    importance_weighted_price = conversion_table.groupby(by='base_asset').agg('sum')
    importance_weighted_price = importance_weighted_price['importance_weighted_price']
    conversion_table['traded_price'] = \
        conversion_table.apply(lambda x: importance_weighted_price.loc[x['base_asset']], axis='columns')
    importance_weighted_bid_price = conversion_table.groupby(by='base_asset').agg('sum')
    importance_weighted_bid_price = importance_weighted_bid_price['importance_weighted_bid_price']
    conversion_table['traded_bid_price'] = \
        conversion_table.apply(lambda x: importance_weighted_bid_price.loc[x['base_asset']], axis='columns')
    importance_weighted_ask_price = conversion_table.groupby(by='base_asset').agg('sum')
    importance_weighted_ask_price = importance_weighted_ask_price['importance_weighted_ask_price']
    conversion_table['traded_ask_price'] = \
        conversion_table.apply(lambda x: importance_weighted_ask_price.loc[x['base_asset']], axis='columns')

    conversion_table['bid_ask_change_percent'] = \
        ((conversion_table['ask_price'] - conversion_table['bid_price']) / conversion_table['ask_price'])
    conversion_table['bid_ask_volume_percent'] = \
        (conversion_table['bid_volume'] / (conversion_table['bid_volume'] + conversion_table['ask_volume']))
    conversion_table[['bid_ask_change_percent', 'bid_ask_volume_percent']] *= 100
    conversion_table['traded_bid_ask_change_percent'] = \
        ((conversion_table['traded_ask_price'] - conversion_table['traded_bid_price']) / \
         conversion_table['traded_ask_price'])
    conversion_table['traded_bid_ask_volume_percent'] = \
        (conversion_table['traded_bid_volume'] / (conversion_table['traded_bid_volume'] + \
                                                  conversion_table['traded_ask_volume']))
    conversion_table[['traded_bid_ask_change_percent', 'traded_bid_ask_volume_percent']] *= 100

    conversion_table = conversion_table[~conversion_table['is_shorted']]
    conversion_table = \
        conversion_table[['symbol', 'base_asset', 'quote_asset', 'is_shorted', 'price_change_percent', 
                          'weighted_average_price', 'open', 'high', 'low', 'close', 'close_shifted', 
                          'last_volume', 'bid_price', 'bid_volume', 'ask_price', 'ask_volume', 
                          'close_time', 'last_ID', 'count', 'rolling_base_volume', 'rolling_quote_volume', 
                          'importance', 'USDT_price', 'rolling_USDT_base_volume', 'rolling_USDT_quote_volume', 
                          'USDT_bid_price', 'USDT_ask_price', 'USDT_bid_volume', 'USDT_ask_volume', 
                          'rolling_traded_volume', 'traded_bid_volume', 'traded_ask_volume', 'traded_price', 
                          'traded_bid_price', 'traded_ask_price', 'bid_ask_change_percent', 
                          'bid_ask_volume_percent', 'traded_bid_ask_change_percent', 
                          'traded_bid_ask_volume_percent']]

    conversion_table = conversion_table.sort_values(by='close_time')
    conversion_table = conversion_table.reset_index(drop=True)
    return conversion_table

conversion_table = get_conversion_table(client=client, exchange_info=exchange_info)
#conversion_table.sort_values(by='rolling_traded_volume', ascending=False).reset_index(drop=True).head(50)
conversion_table

In [1]:
symbol = 'NEBLBUSD'

In [ ]:
import pandas as pd
crypto_output_log_1d = 'crypto_logs/crypto_output_log_1d.txt'
df_1d = pd.read_csv(crypto_output_log_1d, header=[0, 1], index_col=0)
df_1d.index = pd.DatetimeIndex(df_1d.index)
df_1d

In [ ]:
import pandas as pd
crypto_output_log_30s = 'crypto_logs/crypto_output_log_30s.txt'
df_30s = pd.read_csv(crypto_output_log_30s, header=[0, 1], index_col=0)
df_30s.index = pd.DatetimeIndex(df_30s.index)
df_30s

In [ ]:
import pandas as pd
crypto_output_log_15s = 'crypto_logs/crypto_output_log_15s.txt'
df_15s = pd.read_csv(crypto_output_log_15s, header=[0, 1], index_col=0)
df_15s.index = pd.DatetimeIndex(df_15s.index)
df_15s

In [ ]:
df_15s[symbol]

In [2]:
import pandas as pd
crypto_output_log_1min = 'crypto_logs/crypto_output_log_1min.txt'
df_1min = pd.read_csv(crypto_output_log_1min, header=[0, 1], index_col=0)
df_1min.index = pd.DatetimeIndex(df_1min.index)
df_1min

symbol,1INCHBTC,1INCHBUSD,1INCHUSDT,AAVEBNB,AAVEBTC,AAVEBUSD,AAVEETH,AAVEUSDT,ACABTC,ACABUSD,...,ZILBTC,ZILBUSD,ZILETH,ZILEUR,ZILTRY,ZILUSDT,ZRXBTC,ZRXBUSD,ZRXETH,ZRXUSDT
pair,open,open,open,open,open,open,open,open,open,open,...,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume
date,,,,,,,,,,,,,,,,,,,,,
2022-10-24 13:38:00,0.000029,0.567,0.567,0.3212,0.004548,87.6,0.0657,87.5,0.000008,0.1618,...,0.000000,0.000000,0.325595,0.000000,0.000000e+00,5.954437e+03,0.000000,0.0000,0.000000,0.000000e+00
2022-10-24 13:39:00,0.000029,0.568,0.568,0.3218,0.004550,87.7,0.0657,87.7,0.000008,0.1618,...,0.000000,0.000000,0.325595,0.000000,0.000000e+00,6.538788e+03,0.000000,48.1338,0.000000,1.337502e+03
2022-10-24 13:40:00,0.000029,0.568,0.569,0.3221,0.004553,87.8,0.0658,87.8,0.000008,0.1618,...,0.002014,0.000000,0.325595,0.000000,0.000000e+00,7.047066e+03,0.000000,48.1338,0.000000,1.337502e+03
2022-10-24 13:41:00,0.000029,0.569,0.569,0.3221,0.004556,87.9,0.0658,87.8,0.000008,0.1617,...,0.002014,0.000000,0.325595,0.000000,0.000000e+00,7.265759e+03,0.000000,48.1338,0.000000,1.337502e+03
2022-10-24 13:42:00,0.000029,0.569,0.569,0.3221,0.004557,87.9,0.0658,87.9,0.000008,0.1617,...,0.002014,0.000000,0.325595,0.000000,3.872770e+02,7.520786e+03,0.000000,48.1338,0.000000,1.512263e+03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-26 13:44:00,0.000030,0.614,0.614,0.2875,0.004024,83.3,0.0537,83.4,0.000008,0.1707,...,8.551248,400512.524745,152.710580,42190.122133,5.409575e+06,6.069553e+06,4.886703,113249.4197,13.376864,1.197047e+06
2022-10-26 13:45:00,0.000030,0.614,0.614,0.2875,0.004024,83.4,0.0537,83.4,0.000008,0.1707,...,8.538518,396917.326811,152.710580,42190.122133,5.409575e+06,6.070421e+06,4.871550,113193.4871,13.034606,1.195393e+06


In [3]:
df_1min[symbol].tail(20)

pair,open,high,low,close,base_volume,quote_volume,rolling_base_volume,rolling_quote_volume
date,,,,,,,,
2022-10-26 13:29:00,2.053,2.053,2.053,2.053,8.8,18.0664,2395369.1,5.169361e+06
2022-10-26 13:30:00,2.053,2.053,2.053,2.053,0.0,0.0000,2395321.4,5.169266e+06
2022-10-26 13:31:00,2.049,2.051,2.049,2.049,223.0,456.9934,2395467.9,5.169571e+06
2022-10-26 13:32:00,2.049,2.052,2.047,2.047,208.1,426.4800,2395676.0,5.169998e+06
2022-10-26 13:33:00,2.052,2.052,2.052,2.052,217.5,446.3100,2395118.7,5.168906e+06
2022-10-26 13:34:00,2.052,2.052,2.052,2.052,0.0,0.0000,2395060.3,5.168789e+06
2022-10-26 13:35:00,2.052,2.052,2.052,2.052,27.4,56.2248,2394675.2,5.168025e+06
2022-10-26 13:36:00,2.052,2.052,2.052,2.052,0.0,0.0000,2394628.8,5.167933e+06
2022-10-26 13:37:00,2.055,2.058,2.055,2.057,316.6,651.0532,2394945.4,5.168584e+06


In [ ]:
import pandas as pd
crypto_output_log_5min = 'crypto_logs/crypto_output_log_5min.txt'
df_5min = pd.read_csv(crypto_output_log_5min, header=[0, 1], index_col=0)
df_5min.index = pd.DatetimeIndex(df_5min.index)
df_5min

In [ ]:
df_5min[symbol].tail(20)

In [ ]:
df = df_1min.copy()
df

In [5]:
from cryptocurrency.authentication import Cryptocurrency_authenticator
from cryptocurrency.exchange import Cryptocurrency_exchange
from cryptocurrency.conversion_table import get_conversion_table

authenticator = Cryptocurrency_authenticator(use_keys=False, testnet=False)
client = authenticator.spot_client
exchange = Cryptocurrency_exchange(client=client, directory='crypto_logs')
exchange_info = exchange.info

#conversion_table = get_conversion_table(client=client, exchange_info=exchange_info)
#conversion_table.sort_values(by='rolling_traded_volume', ascending=False).reset_index(drop=True).head(50)

In [9]:
from cryptocurrency.conversion import get_shortest_pair_path_between_assets

def convert_ohlcv(from_asset, to_asset, conversion_table, exchange_info):
    shortest_path = get_shortest_pair_path_between_assets(from_asset=from_asset, 
                                                          to_asset=to_asset, 
                                                          exchange_info=exchange_info)
    base_asset, quote_asset = shortest_path[0]
    to_asset = quote_asset if from_asset == base_asset else base_asset
    pair = base_asset + quote_asset
    size = conversion_table[pair].copy()
    size = size.astype(float)
    size['quote_volume'] = size['quote_volume'] / size['close']
    for (base_asset, quote_asset) in shortest_path[1:]:
        to_asset = quote_asset if from_asset == base_asset else base_asset
        size = size.astype(float)
        pair = base_asset + quote_asset
        connection = conversion_table[pair]
        if base_asset == from_asset:
            size['open'] = size['open'] * connection['open']
            size['high'] = size['close'] * connection['high']
            size['low'] = size['close'] * connection['low']
            size['close'] = size['close'] * connection['close']
            size['base_volume'] = size['close'] * connection['base_volume']
            size['quote_volume'] = size['close'] * connection['quote_volume']
        else:
            size['open'] = size['open'] / connection['open']
            size['high'] = size['close'] / connection['high']
            size['low'] = size['close'] / connection['low']
            size['close'] = size['close'] / connection['close']
            size['base_volume'] = size['close'] / connection['base_volume']
            size['quote_volume'] = size['close'] / connection['quote_volume']
        from_asset = to_asset
    return size.rename(columns={'open': 'open_' + to_asset, 
                                'high': 'high_' + to_asset, 
                                'low': 'low_' + to_asset, 
                                'close': 'close_' + to_asset, 
                                'base_volume': 'base_volume_' + to_asset, 
                                'quote_volume': 'quote_volume_' + to_asset})

In [13]:
conversion_table['BTCUSDT']

pair,open,high,low,close,base_volume,quote_volume,rolling_base_volume,rolling_quote_volume
date,,,,,,,,
2022-10-24 13:38:00,19254.02,19260.85,19253.74,19259.95,105.92498,2.039821e+06,105.92498,2.039821e+06
2022-10-24 13:39:00,19259.09,19278.70,19257.04,19276.88,182.71387,3.520396e+06,288.63885,5.560217e+06
2022-10-24 13:40:00,19276.88,19289.80,19274.27,19283.72,246.81135,4.759376e+06,535.45020,1.031959e+07
2022-10-24 13:41:00,19283.71,19288.52,19282.98,19285.27,106.50713,2.054094e+06,641.95733,1.237369e+07
2022-10-24 13:42:00,19286.07,19294.46,19285.00,19293.57,120.72441,2.328919e+06,762.68174,1.470261e+07
...,...,...,...,...,...,...,...,...
2022-10-26 13:44:00,20734.22,20747.70,20732.22,20745.73,179.60033,3.725097e+06,419817.97835,8.585165e+09
2022-10-26 13:45:00,20745.73,20748.92,20729.29,20730.76,184.96839,3.836232e+06,419699.10454,8.582919e+09
2022-10-26 13:46:00,20731.42,20743.72,20728.99,20737.97,119.30018,2.473878e+06,419623.68797,8.581495e+09


In [12]:
from_asset = 'BTC'
to_asset = 'USDT'
conversion_table = df_1min.copy()
df_test = convert_ohlcv(from_asset, to_asset, conversion_table, exchange_info)
df_test

pair,open_USDT,high_USDT,low_USDT,close_USDT,base_volume_USDT,quote_volume_USDT,rolling_base_volume,rolling_quote_volume
date,,,,,,,,
2022-10-24 13:38:00,19254.02,19260.85,19253.74,19259.95,105.92498,105.909962,105.92498,2.039821e+06
2022-10-24 13:39:00,19259.09,19278.70,19257.04,19276.88,182.71387,182.622720,288.63885,5.560217e+06
2022-10-24 13:40:00,19276.88,19289.80,19274.27,19283.72,246.81135,246.807984,535.45020,1.031959e+07
2022-10-24 13:41:00,19283.71,19288.52,19282.98,19285.27,106.50713,106.511054,641.95733,1.237369e+07
2022-10-24 13:42:00,19286.07,19294.46,19285.00,19293.57,120.72441,120.709602,762.68174,1.470261e+07
...,...,...,...,...,...,...,...,...
2022-10-26 13:44:00,20734.22,20747.70,20732.22,20745.73,179.60033,179.559719,419817.97835,8.585165e+09
2022-10-26 13:45:00,20745.73,20748.92,20729.29,20730.76,184.96839,185.050223,419699.10454,8.582919e+09
2022-10-26 13:46:00,20731.42,20743.72,20728.99,20737.97,119.30018,119.292213,419623.68797,8.581495e+09
